In [ ]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET # Facilitando a extração de dados XML
import os 
import shutil
from tqdm import tqdm # Barra de progresso
import yaml
import matplotlib.pyplot as plt 
import torch
import cv2 # Visão computacional
import pytesseract as pt # OCR
%matplotlib inline

In [ ]:
filenames = []

size_props = {
    'height':[],
    'width':[]
}

bounding_box_props = {
    'xmin':[],
    'ymin':[],
    'xmax':[],
    'ymax':[]
}

In [ ]:
# Pegando os dados das anotações
annotations_path = '../input/car-plate-detection/annotations'
for file in tqdm(os.listdir(annotations_path)):
    annotation = ET.parse(os.path.join(annotations_path, file))
    filenames.append(os.path.join(annotations_path, file))
    size = annotation.find('size')
    for name, prop_list in size_props.items():
        prop_value = size.find(name).text
        size_props[name].append(int(prop_value))
    bounding_box = annotation.find('object').find('bndbox')
    for name, prop_list in bounding_box_props.items():
        prop_value = bounding_box.find(name).text
        bounding_box_props[name].append(int(prop_value))

In [ ]:
df = pd.DataFrame({
    'file':filenames,
    'width':size_props['width'],
    'height':size_props['height'],
    'xmin':bounding_box_props['xmin'],
    'ymin':bounding_box_props['ymin'],
    'xmax':bounding_box_props['xmax'],
    'ymax':bounding_box_props['ymax']
})

In [ ]:
df.head()

In [ ]:
# Tornando as anotação compatíveis com o modelo YOLO
df['center_x'] = (df['xmax'] + df['xmin'])/(2*df['width'])
df['center_y'] = (df['ymax'] + df['ymin'])/(2*df['height'])

df['bb_width'] = (df['xmax'] - df['xmin'])/df['width']
df['bb_height'] = (df['ymax'] - df['ymin'])/df['height']

In [ ]:
df.head()

In [ ]:
# Mantendo somente as colunas importantes
yolo_df = df[['file', 'center_x', 'center_y', 'bb_width', 'bb_height']]
# Deixando 70 % dos dados para treinamento, 15% para teste e 15% para validação
test_size = int(0.15 * len(df))

df_train, df_test = train_test_split(yolo_df, test_size=test_size)
df_train, df_val = train_test_split(df_train, test_size=test_size)

In [ ]:
train_path = os.path.join('Images', 'train')
val_path = os.path.join('Images','val')
test_path = os.path.join('Images', 'test')
images_path = '../input/car-plate-detection/images'

if not os.path.exists(train_path):
    os.makedirs(train_path)
    print('Criação de pasta para os dados de treinamento')

if not os.path.exists(val_path):
    os.makedirs(val_path)
    print('Criação de pasta para os dados de validação')

if not os.path.exists(test_path):
    os.makedirs(test_path)
    print('Criação de pasta para os dados de teste')

In [ ]:
print('Movendo as imagens para a pasta de treinamento')
for _, row  in tqdm(df_train.iterrows()):
    annotation_path = row['file']
    image_name = os.path.split(annotation_path)[-1].replace('.xml','')
    image_src = os.path.join(images_path, f'{image_name}.png')
    image_dst = os.path.join(train_path, f'{image_name}.png')
    shutil.copy2(image_src, image_dst)
    label_text = f"0 {row['center_x']} {row['center_y']} {row['bb_width']} {row['bb_height']}"
    with open(os.path.join(train_path, f'{image_name}.txt'), 'w') as f:
        f.write(label_text)
print('Imagens movidas para a pasta de treinamento')

print('Movendo as imagens para a pasta de validação')
for _, row  in tqdm(df_val.iterrows()):
    annotation_path = row['file']
    image_name = os.path.split(annotation_path)[-1].replace('.xml','')
    image_src = os.path.join(images_path, f'{image_name}.png')
    image_dst = os.path.join(val_path, f'{image_name}.png')
    shutil.copy2(image_src, image_dst)
    label_text = f"0 {row['center_x']} {row['center_y']} {row['bb_width']} {row['bb_height']}"
    with open(os.path.join(val_path, f'{image_name}.txt'), 'w') as f:
        f.write(label_text)
print('Imagens movidas para a pasta de validação')

print('Movendo as iamgens para a pasta de teste')
for _, row  in tqdm(df_test.iterrows()):
    annotation_path = row['file']
    image_name = os.path.split(annotation_path)[-1].replace('.xml','')
    image_src = os.path.join(images_path, f'{image_name}.png')
    image_dst = os.path.join(test_path, f'{image_name}.png')
    shutil.copy2(image_src, image_dst)
    label_text = f"0 {row['center_x']} {row['center_y']} {row['bb_width']} {row['bb_height']}"
    with open(os.path.join(test_path, f'{image_name}.txt'), 'w') as f:
        f.write(label_text)
print('Imagens movidas para a pasta de teste')

In [ ]:
# Clone do repositório do YOLO
! git clone https://github.com/ultralytics/yolov5.git

In [ ]:
# Instalando as dependências do YOLO
! pip install -r yolov5/requirements.txt

In [ ]:
# Criação de um arquivo data.yaml que vai ser usado para o YOLO conseguir utilizar os dados do dataset
data = {
    'names':['License Plate'],
    'nc':1,
    'train':os.path.abspath(train_path),
    'val':os.path.abspath(val_path)
}

with open('data.yaml', 'w') as f:
    yaml.dump(data, f)

In [ ]:
# Treinando o modelo YOLO
! python ./yolov5/train.py --data ./data.yaml  --batch-size 8  --epochs 15 --weights yolov5/yolov5s.pt

In [ ]:
# Pegando a última rodada de execução
yolo_path = '/kaggle/working/yolov5/runs/train/'
latest_run = os.listdir(yolo_path)[-1]

# Pegando os melhores pesos 
best_weights = os.path.join(yolo_path, latest_run, 'weights', 'best.pt')

# Carregando o modelo com os melhores pesos treinados
model = torch.hub.load('ultralytics/yolov5', 'custom', best_weights)

In [ ]:
 %matplotlib inline
# Pegando dezesseis imagens aleatórias dos dados de teste
test_files = df_test['file'].apply(lambda x: x.replace('annotations','images').replace('xml','png'))
test_images = np.random.choice(test_files, size=(16,16))
# Vizualizando as predições
fig, ax = plt.subplots(figsize=(16,16), nrows=4, ncols=4)

for i in range(4):
    for j in range(4):
        image = cv2.imread(test_images[i,j])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = model(image,640)
        # Pegando as coordenadas da placa
        results_df = results.pandas().xyxy[0].loc[0]
        x_min = int(results_df['xmin'])
        x_max = int(results_df['xmax'])
        y_min = int(results_df['ymin'])
        y_max = int(results_df['ymax'])
        # Cortando a placa da imagem
        number_plate = image[y_min:y_max,x_min:x_max]
        # Fazendo OCR na imagem
        text = pt.image_to_string(number_plate)
        # Mostrando os resultados
        ax[i,j].set_title(str(text).strip())
        ax[i,j].imshow(np.squeeze(results.render()))
plt.show()

In [ ]:
%matplotlib inline

import seaborn as sns
import glob

In [ ]:
# Redimensionando a imagem para facilitar o processamento
IMAGE_SIZE = 224

In [ ]:
img_dir = "../input/car-plate-detection/images"
data_path = os.path.join(img_dir,'*g')
files = glob.glob(data_path)
files.sort() # Ordenação das imagens em ordem alfabética para combinar com os arquivos xml de anotações
for f1 in files:
    img = cv2.imread(f1)
    img = cv2.resize(img, (IMAGE_SIZE,IMAGE_SIZE))
    X.append(np.array(img))

In [ ]:
from lxml import etree
# Redimensionando as anotações para torná-las compatíveis com o modelo
def resizeannotation(f):
    tree = etree.parse(f)
    for dim in tree.xpath("size"):
        width = int(dim.xpath("width")[0].text)
        height = int(dim.xpath("height")[0].text)
    for dim in tree.xpath("object/bndbox"):
        xmin = int(dim.xpath("xmin")[0].text)/(width/IMAGE_SIZE)
        ymin = int(dim.xpath("ymin")[0].text)/(height/IMAGE_SIZE)
        xmax = int(dim.xpath("xmax")[0].text)/(width/IMAGE_SIZE)
        ymax = int(dim.xpath("ymax")[0].text)/(height/IMAGE_SIZE)
    return [int(xmax), int(ymax), int(xmin), int(ymin)]

In [ ]:
# Pegando as anotações e colocando em uma lista de labels (y)
path = '../input/car-plate-detection/annotations'
text_files = ['../input/car-plate-detection/annotations/'+f for f in sorted(os.listdir(path))]
y=[]
for i in text_files:
    y.append(resizeannotation(i))

In [ ]:
# Transformando os dados em um array numpy
X=np.array(X)
y=np.array(y)

In [ ]:
# Normalização dos dados
X = X / 255
y = y / 255

In [ ]:
# Separando os dados de treinamento,teste e validação
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1)

In [ ]:
# Criando o modelo com Keras
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Conv2D,MaxPooling2D,Dense
model = Sequential()
model.add(Conv2D(64 , (3,3) , input_shape=(224,224,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Dropout(0.1))

model.add(Conv2D(32 , (3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(64, activation="relu"))
model.add(Dense(4, activation="sigmoid"))

model.compile(optimizer='adam',
              loss='mean_squared_error',
             metrics=['accuracy'])

model.summary()

In [ ]:
# Treinando o modelo
train = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=15, batch_size=32, verbose=1)

In [ ]:
# Teste da acurácia do modelo
scores = model.evaluate(X_test, y_test, verbose=0)
print("Score : %.2f%%" % (scores[1]*100))

In [ ]:
# Predição de placas 
y_cnn = model.predict(X_test)

In [ ]:
# Mostrando as placas preditas
plt.figure(figsize=(20,40))
for i in range(0,43) :
    plt.subplot(10,5,i+1)
    plt.axis('off')
    ny = y_cnn[i]*255
    image = cv2.rectangle(X_test[i],(int(ny[0]),int(ny[1])),(int(ny[2]),int(ny[3])),(0, 255, 0))
    plt.imshow(image)